# Step 0
## The REPL

In [1]:
def READ(program:str) -> str:
    return program

In [2]:
def EVAL(line:str) -> str:
    return line

In [3]:
def PRINT(line:str):
    print(line)

In [4]:
def rep(program):
    PRINT(EVAL(READ(program)))

#### Tests

In [113]:
rep(';=>%')

;=>%


# Step 1

In [19]:
# Mal types?

Symbol = str          
List   = list         
Number = (int, float)

#### Test

In [112]:
dgg = Number(9)
type(dgg)

TypeError: 'tuple' object is not callable

In [6]:
def read_str():
    tokens = tokenize(program)
    reader = Reader(tokens)
    read_form(reader)

In [7]:
class Reader:
    def __init__(self, tokens):
        self.tokens = tokens
        self.position = 0
        self.length = len(tokens)
    def nex(self):
        pos = self.position
        current_token = self.tokens[self.position]
        self.position += 1
        return current_token
    def peek(self):
        pos = self.position
        current_token = self.tokens[self.position]
        return current_token

In [92]:
import re

r = r'''[\s,]*(~@|[\[\]{}()'`~^@]|"(?:\\.|[^\\"])*"|;.*|[^\s\[\]{}('"`,;)]*)'''

def tokenize(program:str) -> list:
    tkns = re.split(r, program)
    tkns_list = []
    for t in tkns:
        if t != '':
            tkns_list.append(t) ### ESTO ESTA MAL
    return tkns_list

#### Tests

In [105]:
t = tokenize('( + 2 (* 3 4))')

t

['(', '+', '2', '(', '*', '3', '4', ')', ')']

tokenize

    [\s,]*: Matches any number of whitespaces or commas. This is not captured so it will be ignored and not tokenized. --- ANDA PERO ESTA MAL

    ~@: Captures the special two-characters ~@ (tokenized). ---- ANDA

    [\[\]{}()'`~^@]: Captures any special single character, one of []{}()'`~^@ (tokenized). ---- NO ANDA LOS PONE TODOS JUNTOS

    "(?:\\.|[^\\"])*"?: Starts capturing at a double-quote and stops at the next double-quote unless it was preceded by a backslash in which case it includes it until the next double-quote (tokenized). It will also match unbalanced strings (no ending double-quote) which should be reported as an error. ---- ANDA

    ;.*: Captures any sequence of characters starting with ; (tokenized). ---- ANDA

    [^\s\[\]{}('"`,;)]*: Captures a sequence of zero or more non special characters (e.g. symbols, numbers, "true", "false", and "nil") and is sort of the inverse of the one above that captures special characters (tokenized).


In [106]:
def read_form(tokens:Reader):
    char = tokens.peek()
    result = []
    if char == '(':
        l = read_list(tokens)
        print(l)
        result.append(l)
    else: 
        a = read_atom(char)
        result.append(a)
    return result

 The return value from read_form is a mal data type. If your target language is statically typed then you will need some way for read_form to return a variant or subclass type. For example, if your language is object oriented, then you can define a top level MalType (in types.qx) that all your mal data types inherit from. The MalList type (which also inherits from MalType) will contain a list/array of other MalTypes. If your language is dynamically typed then you can likely just return a plain list/array of other mal types.

In [87]:
def read_list(tokens):
    result = []
    char = tokens.peek()
    while char != ')':
        char = tokens.nex()
        result.append(char)
#    result.append(read_form(tokens))
    return result

# poner algo que corte si te olvidas un parentesis de cierre 

This function will repeatedly call read_form with the Reader object until it encounters a ')' token (if it reach EOF before reading a ')' then that is an error). It accumulates the results into a List type. If your language does not have a sequential data type that can hold mal type values you may need to implement one (in types.qx). Note that read_list repeatedly calls read_form rather than read_atom. This mutually recursive definition between read_list and read_form is what allows lists to contain lists.


In [107]:
def read_atom(token):
    "Numbers become numbers; every other token is a symbol."
    try: return int(token)
    except ValueError:
        try: return float(token)
        except ValueError:
            return Symbol(token)

In [115]:
ghh = read_atom('s')


str

#### Tests

In [108]:
my_str = "(aaaa)"

my_tok = Reader(my_str)
result = read_form(my_tok)

print(result)

['(', 'a', 'a', 'a', 'a', ')']
[['(', 'a', 'a', 'a', 'a', ')']]
